In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.kernel_ridge import KernelRidge
import scipy.stats as stat
np.random.seed(4242)

# General dataset preparation

In [ ]:
df_tr  = pd.read_csv("datasets/DatasetQuinonesFilteredTrain.csv")
df_tst = pd.read_csv("datasets/DatasetQuinonesFilteredTest.csv")

In [ ]:
fnames = [
   'BCUTc-1l',
   'BCUTc-1h',
   'nBondsD2',
   'C1SP1',
   'C3SP2',
   'ECCEN',
   'nHother',
   'ndssC',
   'naasC',
   'ntN',
   'SwHBa',
   'SHdsCH',
   'SHother',
   'SdssC',
   'SaasC',
   'StN',
   'minHother',
   'minaasC',
   'mindO',
   'maxHother',
   'maxaasC',
   'meanI',
   'MAXDN2',
   'MAXDP2',
   'fragC',
   'nRing',
   'topoDiameter',
   'JGI2',
   'TopoPSA',
   'VABC',
   'MW',
   'LUMO']

In [ ]:
X_df_tr     = df_tr[fnames]
y_tr        = df_tr['deltaE_V']
X_df_tst    = df_tst[fnames]
y_tst       = df_tst['deltaE_V']


scaler = StandardScaler()
X_tr = scaler.fit_transform(X_df_tr)
X_tst = scaler.transform(X_df_tst)

# Helper functions

## Hyperparameter optimization

In [ ]:
def plot_cv_score(hp, hp_label, cv_model, xlim=None, ylim=None,xt=None, outf=None, disp_train_score=False, vline=None, l = False):
    plt.figure(figsize=(15,10))
    plt.xlabel(hp_label, fontsize=26)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.ylabel("$R^2$ (average of 5-fold cross-validation)", fontsize=26)
    plt.plot(hp,cv_model.cv_results_['mean_test_score'],linewidth=2,label = 'validation')
    if disp_train_score:
        plt.plot(hp,cv_model.cv_results_['mean_train_score'],linewidth=2, label = 'training')
    if xlim is not None:
        plt.xlim(xlim)
    if ylim is not None:
        plt.ylim(ylim)
    if xt is not None:
        plt.xticks(xt)
    if vline is not None:
        plt.axvline(x=vline, ymin=0, ymax=1, ls='--', c='r')
    if l:
        lgd = plt.legend(bbox_to_anchor=(1.05,.90), fontsize=26,loc='center left')
    if outf is not None:
        if l:
            plt.savefig(outf,bbox_extra_artists=(lgd,), bbox_inches='tight')
        else:
            plt.savefig(outf)
    plt.show()
 
def dump_to_table(hp,hp_label,cv_model):
    dfout = pd.DataFrame()
    dfout["hp_label"]      = hp
    dfout["tr_cv1"]     = cv_model.cv_results_["split0_train_score"]
    dfout["tr_cv2"]     = cv_model.cv_results_["split1_train_score"]
    dfout["tr_cv3"]     = cv_model.cv_results_["split2_train_score"]
    dfout["tr_cv4"]     = cv_model.cv_results_["split3_train_score"]
    dfout["tr_cv5"]     = cv_model.cv_results_["split4_train_score"]
    dfout["tr_cv_mean"] = cv_model.cv_results_["mean_train_score"]
    dfout["tst_cv1"]     = cv_model.cv_results_["split0_test_score"]
    dfout["tst_cv2"]     = cv_model.cv_results_["split1_test_score"]
    dfout["tst_cv3"]     = cv_model.cv_results_["split2_test_score"]
    dfout["tst_cv4"]     = cv_model.cv_results_["split3_test_score"]
    dfout["tst_cv5"]     = cv_model.cv_results_["split4_test_score"]
    dfout["tst_cv_mean"] = cv_model.cv_results_["mean_test_score"]
    return dfout
    

## Test set performance

In [ ]:
def plot_tst_score(X_tst, y_tst, model, outf=None):
    plt.figure(figsize=(15,10))
    y_hat = model.predict(X_tst)
    
    plt.scatter(y_hat, y_tst)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.xlabel("$E_{model}$", fontsize=26)
    plt.ylabel("$E_{DFT}$", fontsize=26)
    modelGraph = LinearRegression()
    modelGraph.fit(y_hat.reshape((-1,1)), y_tst.values.reshape((-1,1)))
    plt.xlim(0,2.6)
    plt.ylim(0,2.6)
    sgn = '+'
    if modelGraph.intercept_[0] < 0:
        sgn='-'
    plt.text(0.2, 2.4, 
             "$E_{DFT}$ = " + "{:.3f}".format(modelGraph.coef_[0][0]) + 
             "$E_{model}$ " + sgn+" {:.3f}".format(np.abs(modelGraph.intercept_)[0]),
            fontsize=26)
    plt.text(0.4, 2.2, 
             "$R^2$ = {:.3f}".format(modelGraph.score(y_hat.reshape((-1,1)), y_tst.values.reshape((-1,1)))),
            fontsize=26)
    xgr = np.arange(0.2, 2.5, 0.1)
    ygr = modelGraph.predict(xgr.reshape(-1,1))
    plt.plot(xgr,ygr,color='r',linewidth=2)
    if outf is not None:
        plt.savefig(outf)
    plt.show()

# Ridge regression

## Hyperparameter tuning

In [ ]:
RidgeRegression = Ridge()
alpha = np.array([0,1e-15,1e-14,1e-13,1e-12,1e-11,1e-10,1e-9,1e-8,1e-6,1e-5,1e-4,1e-3])
alpha = np.concatenate((alpha, np.arange(1e-4,1e-3,1e-4) ,np.arange(1e-2,1e-1,1e-3), np.array([.2,.25,.3,.35,.4,.45,.5,.6,.7,1])))
hyperParameters = {'alpha':alpha}
ridgeRegressor = GridSearchCV(RidgeRegression, hyperParameters, scoring='r2', cv=5, return_train_score=True)
ridgeRegressor.fit(X_tr,y_tr)


In [ ]:
plot_cv_score(alpha, "$\lambda$", ridgeRegressor, disp_train_score=True, vline=.1, l=True,ylim=[.1,1.05], outf="GSRidge.png")
#dump_to_table(alpha,"lambda", ridgeRegressor).to_csv("GSRidge.csv", index=False)

## Performance over test set

In [ ]:
r = Ridge(alpha=0.1)
r.fit(X_tr,y_tr)
plot_tst_score(X_tst, y_tst, r, outf="RidgeTestSet.png")

# Decision tree regression

## Hyperparameter tuning

In [ ]:
DTRegression = DecisionTreeRegressor()
md = np.arange(1,21,1)
hyperParameters = {'max_depth':md}
dtRegressor = GridSearchCV(DTRegression, hyperParameters, scoring='r2', cv=5, return_train_score=True)
dtRegressor.fit(X_tr,y_tr)


In [ ]:
plot_cv_score(md, "Maximal allowed decison tree depth", dtRegressor, xt=np.arange(0,22,2), xlim=[0,21], disp_train_score=True, l=True, vline=3,ylim=[.1,1.05], outf="GSDT.png")
dump_to_table(md,"max_depth", dtRegressor).to_csv("GSST.csv", index=False)

## Performance over test set

In [ ]:
dt = DecisionTreeRegressor(max_depth=3)
dt.fit(X_tr,y_tr)
plot_tst_score(X_tst, y_tst, dt, outf="DTTestSet.png")

## Plot the tree

In [ ]:
plt.figure(figsize = (35,20))
tree.plot_tree(dt, feature_names=fnames, fontsize=20)
plt.savefig("Tree.png")
plt.show()

# RandomForest Regression

## Hyperparameter tuning

In [ ]:
RFRegression = RandomForestRegressor(max_depth=3)
n_est = np.arange(2,100,1)
hyperParameters = {'n_estimators':n_est}
rfRegressor = GridSearchCV(RFRegression, hyperParameters, scoring='r2', cv=5, return_train_score=True)
rfRegressor.fit(X_tr,y_tr)


In [ ]:
plot_cv_score(n_est, "Number of estimators", rfRegressor,disp_train_score=True,l=True,ylim=[.1,1.05], outf="GSRandomForest.png", vline=10)
dump_to_table(n_est,"n_estimators", rfRegressor).to_csv("GSRandomForest.csv", index=False)

## Performance over test set

In [ ]:
rf = RandomForestRegressor(n_estimators=10, max_depth=3)
rf.fit(X_tr,y_tr)
plot_tst_score(X_tst, y_tst, rf, outf="RandomForestTestSet.png")

# ExtraTrees

## Hyperparameter tuning

In [ ]:
ETRegression = ExtraTreesRegressor(max_depth=3)
n_est = np.arange(2,100,1)
hyperParameters = {'n_estimators':n_est}
etRegressor = GridSearchCV(ETRegression, hyperParameters, scoring='r2', cv=5, return_train_score=True)
etRegressor.fit(X_tr,y_tr)


In [ ]:
plot_cv_score(n_est, "Number of estimators", etRegressor,disp_train_score=True,l=True,ylim=[.1,1.05],outf='GSExtraTrees.png', vline=15)
dump_to_table(n_est,"n_estimators", etRegressor).to_csv("GSExtraTrees.csv", index=False)

## Performance over test set

In [ ]:
et = ExtraTreesRegressor(n_estimators=15, max_depth=3)
et.fit(X_tr,y_tr)
plot_tst_score(X_tst, y_tst, et, outf="ExtraTreesTestSet.png")

# GradientBoosting

## Hyperparameter tuning

In [ ]:
GBRegression = GradientBoostingRegressor()
n_est = np.arange(2,100,1)
lr = np.array([0.01,0.05,0.1,0.5])
hyperParameters = {'n_estimators':n_est,
                   'learning_rate':lr}
gbRegressor = GridSearchCV(GBRegression, hyperParameters, scoring='r2', cv=5, return_train_score=True)
gbRegressor.fit(X_tr,y_tr)


### Grid search results to table

In [ ]:
dfout = pd.DataFrame()
cv_model = gbRegressor
dfout["n_est"]      = cv_model.cv_results_['param_n_estimators']
dfout["learning rate"] = cv_model.cv_results_['param_learning_rate']
dfout["tr_cv1"]     = cv_model.cv_results_["split0_train_score"]
dfout["tr_cv2"]     = cv_model.cv_results_["split1_train_score"]
dfout["tr_cv3"]     = cv_model.cv_results_["split2_train_score"]
dfout["tr_cv4"]     = cv_model.cv_results_["split3_train_score"]
dfout["tr_cv5"]     = cv_model.cv_results_["split4_train_score"]
dfout["tr_cv_mean"] = cv_model.cv_results_["mean_train_score"]
dfout["tst_cv1"]     = cv_model.cv_results_["split0_test_score"]
dfout["tst_cv2"]     = cv_model.cv_results_["split1_test_score"]
dfout["tst_cv3"]     = cv_model.cv_results_["split2_test_score"]
dfout["tst_cv4"]     = cv_model.cv_results_["split3_test_score"]
dfout["tst_cv5"]     = cv_model.cv_results_["split4_test_score"]
dfout["tst_cv_mean"] = cv_model.cv_results_["mean_test_score"]
dfout.to_csv("GradientBoosting.csv", index = False)

### Grid search to figure

In [ ]:
lr001tr = dfout[dfout['learning rate'] == 0.01]["tr_cv_mean"]
lr005tr = dfout[dfout['learning rate'] == 0.05]["tr_cv_mean"]
lr01tr = dfout[dfout['learning rate'] == 0.1]["tr_cv_mean"]
lr05tr = dfout[dfout['learning rate'] == 0.5]["tr_cv_mean"]

lr001tst = dfout[dfout['learning rate'] == 0.01]["tst_cv_mean"]
lr005tst = dfout[dfout['learning rate'] == 0.05]["tst_cv_mean"]
lr01tst = dfout[dfout['learning rate'] == 0.1]["tst_cv_mean"]
lr05tst = dfout[dfout['learning rate'] == 0.5]["tst_cv_mean"]

In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("Number of estimators", fontsize=26)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.ylabel("$R^2$ (average of 5-fold cross-validation)", fontsize=26)
m=10
plt.plot(n_est, lr001tst, label = "LR = 0.01, validation",linewidth=0,marker='d', c='b', markersize=m)
plt.plot(n_est, lr001tr,  label = "LR = 0.01, training",linewidth=0, marker='d', c='b', fillstyle='none', markersize=m)#, linestyle='none')
plt.plot(n_est, lr005tst, label = "LR = 0.05, validation",linewidth=0,marker='o', c='r', markersize=m)
plt.plot(n_est, lr005tr,  label = "LR = 0.05, training",linewidth=0, marker='o', c='r', fillstyle='none', markersize=m)
plt.plot(n_est, lr01tst,  label = "LR = 0.10, validation",linewidth=0, marker='X', c='g', markersize=m)
plt.plot(n_est, lr01tr,  label = "LR = 0.10, training",linewidth=0, marker='x', c='g')#, fillstyle='none', markersize=m)
plt.plot(n_est, lr05tst,  label = "LR = 0.50, validation",linewidth=0, marker='^', c='black', markersize=m)
plt.plot(n_est, lr05tr,   label = "LR = 0.50, training",linewidth=0, marker='^', c='black',fillstyle='none', markersize=m )
plt.axvline(x=50, ymin=0, ymax=1, ls='--', c='r')
lgd = plt.legend(bbox_to_anchor=(1.05,1.0), fontsize=26)
plt.savefig("GSGradientBoosting.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

## Performance over test set

In [ ]:
gb = GradientBoostingRegressor(n_estimators=50, learning_rate=.05)
gb.fit(X_tr,y_tr)
plot_tst_score(X_tst, y_tst, gb, outf="GradientBoostingTestSet.png")